# 🏥 Mechano-Velocity: Notebook 04 - Training & Validation

**Clinical Scoring and Model Validation**

This notebook:
1. Generates clinical risk scores (MTS, Metastatic Risk, Immune Exclusion)
2. Validates model predictions against histology
3. Runs ablation studies
4. Stores results in database
5. (Optional) GNN training for advanced physics-informed learning

---

## Clinical Metrics

**Mechano-Therapeutic Score (MTS):**
$$MTS = \frac{\text{T-cell Infiltration Flux}}{\text{Cancer Metastasis Flux}}$$

| MTS Range | Classification | Recommendation |
|-----------|---------------|----------------|
| > 2.0 | Hot / Leaky | Standard Immunotherapy |
| 0.5 - 2.0 | Intermediate | Consider Combination |
| < 0.5 | Cold / Trapped | Anti-fibrotic + Immunotherapy |

## 1. Setup

In [ ]:
# Check environment
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    %cd /content/mechano-velocity

In [ ]:
# Core imports
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

sc.settings.verbosity = 2
sc.settings.set_figure_params(dpi=100, frameon=False, figsize=(8, 8))

In [ ]:
# Import project modules
from pathlib import Path
sys.path.insert(0, '.')

from mechano_velocity import (
    Config, ClinicalScorer, Visualizer, DatabaseManager
)

PROJECT_ROOT = Path('.').resolve()
print(f"Project: {PROJECT_ROOT}")

## 2. Load Data

In [ ]:
# Load configuration
config = Config()
config.output_dir = PROJECT_ROOT / "output"

# Load velocity-corrected data
adata_path = config.output_dir / 'velocity_corrected_adata.h5ad'

if adata_path.exists():
    adata = sc.read_h5ad(adata_path)
    print(f"Loaded: {adata.shape}")
else:
    raise FileNotFoundError("Please run 03_Graph_Simulation.ipynb first.")

In [ ]:
# Verify required fields
required = ['resistance', 'velocity_magnitude']
missing = [r for r in required if r not in adata.obs.columns]

if missing:
    raise ValueError(f"Missing required fields: {missing}")

print("\n✅ All required fields present")
print(f"  obsm keys: {list(adata.obsm.keys())}")

## 3. Initialize Database

In [ ]:
# Initialize database for storing results
db = DatabaseManager(config=config)

print(f"Database initialized: {db.db_path}")

In [ ]:
# Start analysis run
run_id = db.start_analysis_run(
    sample_id=config.dataset_name,
    n_spots=adata.n_obs,
    n_genes=adata.n_vars,
    config_dict=config.to_dict(),
    notes="Full pipeline analysis with clinical scoring"
)

print(f"\n📊 Analysis Run ID: {run_id}")

## 4. Generate Clinical Report

In [ ]:
# Initialize clinical scorer
scorer = ClinicalScorer(config)

In [ ]:
# Generate clinical report
report = scorer.generate_report(
    adata,
    sample_id=config.dataset_name,
    tumor_cluster=None,  # Auto-detect from markers
    tcell_threshold=0.3
)

In [ ]:
# View the clinical report
print(report.to_text())

In [ ]:
# Save report to files
scorer.save_report(config.output_dir / 'clinical_report.txt', format='txt')
scorer.save_report(config.output_dir / 'clinical_report.json', format='json')

In [ ]:
# Store report in database
report_id = db.save_clinical_report(run_id, report.to_dict())
print(f"Report saved to database with ID: {report_id}")

## 5. Visualize Clinical Findings

In [ ]:
# Initialize visualizer
viz = Visualizer(config)

In [ ]:
# Cell type spatial plot
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Tumor regions
if 'is_tumor' in adata.obs.columns:
    sc.pl.spatial(adata, color='is_tumor', ax=axes[0], show=False,
                  title='Tumor Regions', palette={True: 'red', False: 'lightgray'})
else:
    axes[0].text(0.5, 0.5, 'No tumor markers', ha='center', va='center')
    axes[0].set_title('Tumor Regions')

# T-cell regions
if 'is_tcell' in adata.obs.columns:
    sc.pl.spatial(adata, color='is_tcell', ax=axes[1], show=False,
                  title='T-cell Regions', palette={True: 'blue', False: 'lightgray'})
else:
    axes[1].text(0.5, 0.5, 'No T-cell markers', ha='center', va='center')
    axes[1].set_title('T-cell Regions')

# Boundary regions
if 'is_boundary' in adata.obs.columns:
    sc.pl.spatial(adata, color='is_boundary', ax=axes[2], show=False,
                  title='Tumor Boundary', palette={True: 'orange', False: 'lightgray'})
else:
    axes[2].text(0.5, 0.5, 'No boundary defined', ha='center', va='center')
    axes[2].set_title('Tumor Boundary')

plt.tight_layout()
plt.savefig(config.output_dir / 'cell_type_regions.png', dpi=150)
plt.show()

In [ ]:
# Clinical scores visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Score bars
scores = [
    ('Metastatic Risk', report.metastatic_risk_score),
    ('Immune Exclusion', report.immune_exclusion_score),
    ('MTS', report.mechano_therapeutic_score)
]

for i, (name, score) in enumerate(scores):
    color = 'steelblue' if i < 2 else ('green' if score > 2 else 'red' if score < 0.5 else 'orange')
    axes[i].barh([0], [score], color=color, height=0.5)
    axes[i].set_xlim(0, max(score * 1.5, 1))
    axes[i].set_xlabel(name, fontsize=12)
    axes[i].set_yticks([])
    axes[i].set_title(f'{name}: {score:.4f}', fontsize=14)

plt.tight_layout()
plt.savefig(config.output_dir / 'clinical_scores.png', dpi=150)
plt.show()

print(f"\n🏥 CLINICAL CLASSIFICATION: {report.risk_category}")

## 6. Validation Studies

In [ ]:
# Validation 1: Resistance-Velocity Correlation
# Hypothesis: Higher resistance should correlate with lower velocity

from scipy import stats

resistance = adata.obs['resistance'].values
velocity = adata.obs['velocity_magnitude'].values

correlation, p_value = stats.pearsonr(resistance, velocity)

print("\n📊 VALIDATION 1: Resistance-Velocity Correlation")
print(f"  Pearson r: {correlation:.4f}")
print(f"  P-value: {p_value:.2e}")
print(f"  Expected: Negative correlation (high R → low V)")
print(f"  Result: {'✅ PASS' if correlation < 0 else '⚠️ CHECK'}")

# Log to database
db.add_validation_log(
    run_id=run_id,
    validation_type='resistance_velocity_correlation',
    expected='r < 0',
    actual=f'r = {correlation:.4f}',
    passed=correlation < 0,
    notes=f'p-value: {p_value:.2e}'
)

In [ ]:
# Visualize correlation
fig, ax = plt.subplots(figsize=(8, 6))

ax.scatter(resistance, velocity, alpha=0.3, s=10, c='steelblue')

# Fit line
z = np.polyfit(resistance, velocity, 1)
p = np.poly1d(z)
x_line = np.linspace(resistance.min(), resistance.max(), 100)
ax.plot(x_line, p(x_line), 'r--', linewidth=2, label=f'r = {correlation:.3f}')

ax.set_xlabel('Resistance', fontsize=12)
ax.set_ylabel('Velocity Magnitude', fontsize=12)
ax.set_title('Resistance vs Velocity Correlation', fontsize=14)
ax.legend()

plt.tight_layout()
plt.savefig(config.output_dir / 'validation_correlation.png', dpi=150)
plt.show()

In [ ]:
# Validation 2: Wall vs Non-Wall Velocity Comparison
# Hypothesis: Spots in "wall" regions should have lower velocity

wall_mask = adata.obs['resistance_category'] == 'wall'
fluid_mask = adata.obs['resistance_category'] == 'fluid'

if wall_mask.sum() > 0 and fluid_mask.sum() > 0:
    wall_velocity = velocity[wall_mask]
    fluid_velocity = velocity[fluid_mask]
    
    # T-test
    t_stat, t_pvalue = stats.ttest_ind(wall_velocity, fluid_velocity)
    
    print("\n📊 VALIDATION 2: Wall vs Fluid Velocity")
    print(f"  Wall mean velocity: {wall_velocity.mean():.4f}")
    print(f"  Fluid mean velocity: {fluid_velocity.mean():.4f}")
    print(f"  T-statistic: {t_stat:.4f}")
    print(f"  P-value: {t_pvalue:.2e}")
    print(f"  Expected: Wall < Fluid")
    print(f"  Result: {'✅ PASS' if wall_velocity.mean() < fluid_velocity.mean() else '⚠️ CHECK'}")
    
    # Log to database
    db.add_validation_log(
        run_id=run_id,
        validation_type='wall_vs_fluid_velocity',
        expected='wall < fluid',
        actual=f'wall={wall_velocity.mean():.4f}, fluid={fluid_velocity.mean():.4f}',
        passed=wall_velocity.mean() < fluid_velocity.mean(),
        notes=f'p-value: {t_pvalue:.2e}'
    )
else:
    print("\n⚠️ Not enough spots in wall/fluid categories for comparison")

In [ ]:
# Visualize wall vs fluid
fig, ax = plt.subplots(figsize=(8, 6))

categories = ['wall', 'normal', 'fluid']
velocities = []
for cat in categories:
    mask = adata.obs['resistance_category'] == cat
    if mask.sum() > 0:
        velocities.append(velocity[mask])
    else:
        velocities.append(np.array([0]))

ax.boxplot(velocities, labels=categories)
ax.set_xlabel('Resistance Category', fontsize=12)
ax.set_ylabel('Velocity Magnitude', fontsize=12)
ax.set_title('Velocity by Resistance Category', fontsize=14)

plt.tight_layout()
plt.savefig(config.output_dir / 'validation_boxplot.png', dpi=150)
plt.show()

## 7. Ablation Study

In [ ]:
# Test: What if we ignore resistance entirely?
# Compare corrected vs uncorrected velocity distributions

print("\n📊 ABLATION: Effect of Resistance Correction")
print("="*50)

# With correction (current)
corrected_mag = adata.obs['velocity_magnitude'].values

# Calculate what uncorrected would look like
# (uniform velocity based on spatial distance only)
from sklearn.neighbors import NearestNeighbors
coords = adata.obsm['spatial']
nbrs = NearestNeighbors(n_neighbors=7)
nbrs.fit(coords)
distances, indices = nbrs.kneighbors(coords)

# Uncorrected: average distance to neighbors (uniform)
uncorrected_mag = distances[:, 1:].mean(axis=1)  # Exclude self
uncorrected_mag = uncorrected_mag / uncorrected_mag.max()  # Normalize

print(f"  Corrected mean: {corrected_mag.mean():.4f}")
print(f"  Uncorrected mean: {uncorrected_mag.mean():.4f}")
print(f"  Variance reduction: {1 - corrected_mag.var()/uncorrected_mag.var():.2%}")

In [ ]:
# Visualize ablation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution comparison
axes[0].hist(uncorrected_mag, bins=50, alpha=0.5, label='Uncorrected', color='blue')
axes[0].hist(corrected_mag, bins=50, alpha=0.5, label='Corrected', color='red')
axes[0].set_xlabel('Velocity Magnitude')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Velocity Distribution: Corrected vs Uncorrected')
axes[0].legend()

# Scatter
axes[1].scatter(uncorrected_mag, corrected_mag, alpha=0.3, s=10)
axes[1].plot([0, 1], [0, 1], 'r--', label='y = x')
axes[1].set_xlabel('Uncorrected Velocity')
axes[1].set_ylabel('Corrected Velocity')
axes[1].set_title('Effect of Resistance Correction')
axes[1].legend()

plt.tight_layout()
plt.savefig(config.output_dir / 'ablation_study.png', dpi=150)
plt.show()

## 8. Store Spot-Level Data

In [ ]:
# Save detailed spot data to database
n_spots = db.save_spot_data(run_id, adata)
print(f"Saved {n_spots} spots to database")

In [ ]:
# Mark analysis as complete
db.complete_analysis_run(run_id, status='completed')
print(f"\n✅ Analysis run {run_id} marked as complete")

## 9. Query Database

In [ ]:
# View all analysis runs
runs = db.get_analysis_runs(limit=10)
print("\n📋 Recent Analysis Runs:")
for run in runs:
    print(f"  Run {run['id']}: {run['sample_id']} ({run['status']}) - {run['run_timestamp']}")

In [ ]:
# View clinical reports
reports = db.get_clinical_reports(run_id=run_id)
if reports:
    print("\n🏥 Clinical Reports:")
    for r in reports:
        print(f"  Report {r['id']}:")
        print(f"    MTS: {r['mts_score']:.4f}")
        print(f"    Risk: {r['risk_category']}")

In [ ]:
# Export spot data to CSV for external analysis
csv_path = config.output_dir / f'run_{run_id}_spots.csv'
db.export_to_csv(run_id, csv_path)

## 10. (Optional) GNN Training

For more advanced physics-informed learning, you can train a GNN.
This section is optional and requires PyTorch.

In [ ]:
# Check for PyTorch
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    HAS_TORCH = True
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
except ImportError:
    HAS_TORCH = False
    print("PyTorch not available - skipping GNN training")

In [ ]:
if HAS_TORCH:
    try:
        from torch_geometric.nn import GCNConv, SAGEConv
        from torch_geometric.data import Data
        HAS_PYG = True
        print("PyTorch Geometric available")
    except ImportError:
        HAS_PYG = False
        print("PyTorch Geometric not available")
else:
    HAS_PYG = False

In [ ]:
if HAS_TORCH and HAS_PYG:
    # Load PyG data
    pyg_path = config.output_dir / 'spatial_graph.pt'
    
    if pyg_path.exists():
        data = torch.load(pyg_path)
        print(f"Loaded PyG data: {data}")
        
        # Simple GNN for resistance prediction
        class ResistanceGNN(nn.Module):
            def __init__(self, in_channels, hidden_channels):
                super().__init__()
                self.conv1 = SAGEConv(in_channels, hidden_channels)
                self.conv2 = SAGEConv(hidden_channels, hidden_channels)
                self.linear = nn.Linear(hidden_channels, 1)
                
            def forward(self, x, edge_index):
                x = F.relu(self.conv1(x, edge_index))
                x = F.dropout(x, p=0.2, training=self.training)
                x = F.relu(self.conv2(x, edge_index))
                x = torch.sigmoid(self.linear(x))
                return x.squeeze()
        
        print("\nGNN model defined. Ready for training.")
        print("Note: Full training requires labeled data/ground truth.")
    else:
        print(f"PyG data not found at {pyg_path}")
        print("Run notebook 03 to generate it.")

## 11. Final Summary

In [ ]:
# Final summary
print("="*60)
print("MECHANO-VELOCITY ANALYSIS COMPLETE")
print("="*60)
print(f"\nSample: {config.dataset_name}")
print(f"Spots analyzed: {adata.n_obs}")
print(f"Analysis Run ID: {run_id}")
print(f"\nCLINICAL RESULTS:")
print(f"  Metastatic Risk Score: {report.metastatic_risk_score:.4f}")
print(f"  Immune Exclusion Score: {report.immune_exclusion_score:.4f}")
print(f"  Mechano-Therapeutic Score: {report.mechano_therapeutic_score:.4f}")
print(f"\n  Classification: {report.risk_category}")
print(f"\n  Recommendation: {report.therapeutic_recommendation}")
print(f"\nOutput files saved to: {config.output_dir}")
print("="*60)

## Summary

✅ Generated clinical risk scores (MTS, Metastatic Risk, Immune Exclusion)  
✅ Identified tumor, T-cell, and boundary regions  
✅ Validated resistance-velocity correlation  
✅ Performed ablation study  
✅ Stored all results in database  
✅ (Optional) Set up GNN training framework  

**The Mechano-Velocity pipeline is complete!**

For production use:
1. Download trained models from Colab
2. Run inference locally using the `mechano_velocity` package
3. Query the database for historical comparisons